# WINNING HORSE - Limpieza de datos

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime, time

## Datasets

In [2]:
df2022 = pd.read_csv("./../data/Results_Global_2020_winnersTop_nuevo.csv")
df2021 = pd.read_csv("./../data/Results_Global_2021_winnersTop_nuevo.csv")
df2020 = pd.read_csv("./../data/Results_Global_2022_winnersTop_nuevo.csv")
dfResults = pd.concat([df2020, df2021, df2022], ignore_index=True)

## Limpieza de datos - Primera parte
**Opción A**: Eliminar caballos que no corrieron

**Opción B**: Limitar las muestras a carreras de un rango de edad concreto

**Opción C**: Limitar las muestras por sexo

**Opción D**: Limitar las muestras a carreras de eventos principales

**Opción E**: Limitar las muestras a carreras con información de evento principal

In [3]:
#df.drop(df[df["STATUS"] != "Runner"].index, axis=0, inplace=True)
#df.drop(df[df["AGELIMIT"] != "4YO+"].index, axis=0, inplace=True)
#df.drop(df[df["SEXTYPE"] != "GELDING"].index, axis=0, inplace=True)
#df.drop(df[df["MAJOREVENT"] != 1].index, axis=0, inplace=True)
dfResults.dropna(subset=["MAJOREVENT"], inplace=True)


Se limpian las columnas que no se van a utilizar en el análisis

In [4]:
dropColumns = [
    "STATUS",
    "AGELIMIT",
    "SEXTYPE",
    "LATITUDE",
    "LONGITUDE",
    "YEAR",
    "COUNTY",
    "GROUNDTEXT",
    "LEADINGEARNERHORSE",
    "ENTRYTYPE",
    "PRIZEAMOUNT",
    "POSITIONFINISHTIME",
    "RESULTFINISHPOS",
    "TOTALOWNERPRIZEMONEYWON",
    "ownername",
    "TRAINERSTYLE",
    "LICENCETYPE",
    "LOCATION",
    "AGEYEAR"
]
dfResults.drop(dropColumns, axis=1, inplace=True)
print(dfResults.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14996 entries, 0 to 64932
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   REGION                14996 non-null  object 
 1   TRACKHANDEDNESS       14996 non-null  object 
 2   RACEDATE              14996 non-null  object 
 3   RACETIME              14996 non-null  object 
 4   DISTANCE              14996 non-null  int64  
 5   TIER                  14996 non-null  object 
 6   MAJOREVENT            14996 non-null  float64
 7   GROUND                13883 non-null  float64
 8   GOINGSTICK            11102 non-null  float64
 9   WEATHER               2752 non-null   object 
 10  DIVISIONSEQUENCE      14996 non-null  int64  
 11  NAME                  14996 non-null  object 
 12  DATEOFBIRTH           14996 non-null  object 
 13  CLOTHNUMBER           14996 non-null  int64  
 14  WEIGHTVALUE           14996 non-null  int64  
 15  BETTINGRATIO       

## Características del dataset

Reemplazar los ratios de divisiones por 0 y evaluamos para convertirlos en numéricos y eliminamos la columna de literales

In [5]:
dfResults["EVAL_BETTINGRATIO"] = dfResults["BETTINGRATIO"].replace({"0/0":"0/1"}).apply(eval)

Cambiar las fechas a formato datetime para poder operar con ellas, y combinación de date y time en una sola variable

In [6]:
dfResults["DATEOFBIRTH"] = pd.to_datetime(dfResults["DATEOFBIRTH"])
dfResults["RACEDATE"] = pd.to_datetime(dfResults["RACEDATE"] + ' ' + dfResults["RACETIME"])

Creación del dato EdadYears que refleja la edad exacta en años del caballo el día de la carrera

In [7]:
edades = dfResults["RACEDATE"] - dfResults["DATEOFBIRTH"]
dfResults["EdadYears"] = edades / np.timedelta64(1, 'Y')

Obtenemos las estaciones del año por carrera, nos permite generalizar más que el día y el mes, por si un caballo tiene mejor rendimiento en estaciones cálidas o frías

In [8]:
anio = 2000;
datesSeasons = [
    ("Winter", (date(anio, 1, 1), date(anio, 3, 19))),
    ("Spring", (date(anio, 3, 20), date(anio, 6, 20))),
    ("Summer", (date(anio, 6, 21), date(anio, 9, 22))),
    ("Fall", (date(anio, 9, 23), date(anio, 12, 21))),
    ("Winter", (date(anio, 12, 22), date(anio, 12, 31))),
]
def get_season(now):
    now = now.date()
    now = now.replace(year=anio)
    return next(a for a, (start, end) in datesSeasons if start <= now <= end)

dfResults["SEASONS"] = dfResults["RACEDATE"].apply(lambda x: get_season(x))

Obtenemos el rango de horas según la hora de salida (mañana, mediodia, tarde y noche)

In [9]:
def horario(hora):
    if hora.time() >= time(7) and hora.time() < time(11):
        return "Morning"
    elif hora.time() >= time(11) and hora.time() < time(15):
        return "Midday"
    if hora.time() >= time(15) and hora.time() < time(19):
        return "Afternoon"
    else:
        return "Night"

dfResults["HORARIO"] = dfResults["RACEDATE"].apply(lambda x: horario(x))

Sustituir valores nulos de Major event y de Ground
- ***MAJOR EVENT*** : Los eventos con valor nulo se catalogan con valor 0 (no importantes)
- ***GROUND*** : La condición de terreno con valor nulo se cataloga con la media de los valores conocidos
- ***GOING STICK*** : La condición de agarre del terreno con valor nulo se cataloga con la media de los valores conocidos

In [10]:
# Rellenamos los valores de MajorEvent = NaN con valor 0
dfResults["MAJOREVENT"].fillna(0, inplace=True)

#Rellenar los valores de Ground = NaN con la media de la columna
dfResults["GROUND"].fillna(dfResults["GROUND"].mean(), inplace=True)

#Rellenar los valores de GoingStick = NaN con la media de la columna
dfResults["GOINGSTICK"].fillna(dfResults["GOINGSTICK"].mean(), inplace=True)

### Manga de caballos
Creamos un identificador único para las mangas, con el fin de sacar el mínimo betting ratio por manga y así calcular los favoritos
*(Funciones de larga duración)*

In [11]:
#Por cada fecha, buscamos las horas y por cada hora, buscamos la manga y por cada manga añadimos un identificador para identificar la manga
id_manga = 0
fechas = dfResults["RACEDATE"].unique()
for fecha in fechas:
    mangas = dfResults[dfResults["RACEDATE"] == fecha]["DIVISIONSEQUENCE"].unique()
    for manga in mangas:
        dfResults.loc[(dfResults["RACEDATE"] == fecha) & (dfResults["DIVISIONSEQUENCE"] == manga), "IDMANGA"] = id_manga
        id_manga += 1

#Creamos una columna con el valor mínimo del bettingRatio por cada manga y el mejor tiempo
for manga in dfResults["IDMANGA"]:
    dfResults.loc[dfResults["IDMANGA"] == manga, "MIN_BETTINGRATIO"] = dfResults[dfResults["IDMANGA"] == manga]["EVAL_BETTINGRATIO"].min()

#Asignamos la columna de favorito a True de la carrera si el BettingRatio es igual al mínimo
dfResults["FAVORITE"] = dfResults["EVAL_BETTINGRATIO"] ==  dfResults["MIN_BETTINGRATIO"]

### WEATER
Comprobamos la variable WEATHER sobre el número de nulos, por si aporta suficiente valor. Dado que el número de nulos es muy elevado, se decide eliminar la columna en vez de completar los nulos.

In [12]:
# obtener el porcentaje de nulos de la columna Weather
print("Porcentaje de nulos de la columna WEATHER: ", dfResults["WEATHER"].isnull().sum() / len(dfResults["WEATHER"]) * 100, "%")

Porcentaje de nulos de la columna WEATHER:  81.64843958388903 %


## Limpieza de datos - Segunda parte

In [13]:
dropColumns = [
    # Eliminamos las columnas que no necesitamos tras el cálculo de la nueva manga
    "EVAL_BETTINGRATIO",
    "MIN_BETTINGRATIO",
    "RACEDATE",
    "IDMANGA",
    "DIVISIONSEQUENCE",

    # Dado que el número de nulos es muy elevado, se decide eliminar la columna en vez de completar los nulos
    "WEATHER",

    # Eliminamos las columnas que no nos aportan información
    "BETTINGRATIO",
    "RACETIME",
    "DATEOFBIRTH",
]
dfResults.drop(dropColumns, axis=1, inplace=True)
print(dfResults.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14996 entries, 0 to 64932
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   REGION                14996 non-null  object 
 1   TRACKHANDEDNESS       14996 non-null  object 
 2   DISTANCE              14996 non-null  int64  
 3   TIER                  14996 non-null  object 
 4   MAJOREVENT            14996 non-null  float64
 5   GROUND                14996 non-null  float64
 6   GOINGSTICK            14996 non-null  float64
 7   NAME                  14996 non-null  object 
 8   CLOTHNUMBER           14996 non-null  int64  
 9   WEIGHTVALUE           14996 non-null  int64  
 10  RACECRITERIARACETYPE  14996 non-null  object 
 11  ENTRYNAME             14996 non-null  object 
 12  CHAMPIONSHIPTYPE      14996 non-null  object 
 13  FULLNAME              14996 non-null  object 
 14  TOP1                  14996 non-null  int64  
 15  TOP3               

### Dummies
Generamos Campos Dummies para las variables categóricas

In [14]:
dummiesColumns = ["TRACKHANDEDNESS", "RACECRITERIARACETYPE", "SEASONS", "HORARIO"]
dummiesColumnsNames = ["TrackHandedness", "RaceCriteria", "Seasons", "Schedule"]
dummies = pd.get_dummies(dfResults[dummiesColumns], dummiesColumnsNames, drop_first=True)
dfResults = pd.concat([dfResults, dummies], axis=1)
dfResults.drop(dummiesColumns, axis=1, inplace=True)
dfResults

,REGION,DISTANCE,TIER,MAJOREVENT,GROUND,GOINGSTICK,NAME,CLOTHNUMBER,WEIGHTVALUE,ENTRYNAME,...,EdadYears,FAVORITE,TrackHandedness_Left-Handed,TrackHandedness_Right-Handed,RaceCriteria_JUMP,Seasons_Spring,Seasons_Summer,Seasons_Winter,Schedule_Midday,Schedule_Night
0,SOUTH,2646,-100,1.0,5.188576,7.177653,Sonny Liston (IRE),12,128,Tom Marquand,...,3.344867,False,1,0,0,1,0,0,0,0
1,SOUTH,2646,-100,1.0,5.188576,7.177653,Grand Alliance (IRE),5,128,Daniel Tudhope,...,3.180592,False,1,0,0,1,0,0,0,0
2,SOUTH,2646,-100,1.0,5.188576,7.177653,Desert Crown (GB),2,128,Richard Kingscote,...,3.259992,True,1,0,0,1,0,0,0,0
3,SOUTH,2646,-100,1.0,5.188576,7.177653,Hoo Ya Mal (GB),6,128,David Probert,...,3.136786,False,1,0,0,1,0,0,0,0
4,SOUTH,2646,-100,1.0,5.188576,7.177653,Grand Alliance (IRE),5,128,Daniel Tudhope,...,3.180592,False,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64928,NORTH,1100,-100,0.0,4.000000,7.000000,Murbih (IRE),10,126,Cam Hardie,...,3.331025,False,0,1,0,0,1,0,0,0
64929,NORTH,1100,-100,0.0,4.000000,7.000000,James Watt (IRE),1,142,Ryan Sexton,...,6.411171,False,0,1,0,0,1,0,0,0
64930,NORTH,1100,-100,0.0,4.000000,7.000000,Melody King (IRE),5,135,Harrison Shaw,...,5.389931,False,0,1,0,0,1,0,0,0
64931,NORTH,1636,-100,0.0,3.000000,7.200000,Liberation Point (IRE),1,140,Frederick Larson,...,5.444746,True,0,1,0,0,1,0,0,0


Comprobamos que los valores objetivo solo tienen dos valores 1 o 0

In [15]:
dfResults["TOP1"].value_counts()

0    13284
1     1712
Name: TOP1, dtype: int64

In [16]:
dfResults["TOP3"].value_counts()

0    9092
1    5904
Name: TOP3, dtype: int64

In [17]:
dfResults["TOP5"].value_counts()

1    9141
0    5855
Name: TOP5, dtype: int64

Renombramos las columnas para un mejor entendimiento

In [18]:
dfResults.rename(columns={'REGION': 'Region',
                   'DISTANCE': 'Distance',
                   'TIER': 'Category',
                   'MAJOREVENT': 'MajorEvent',
                   'GROUND': 'GroundCondition',
                   'GOINGSTICK': 'Stick',
                   'NAME': 'HorseName',
                   'CLOTHNUMBER': 'StartingStall',
                   'WEIGHTVALUE': 'Weight',
                   'ENTRYNAME': 'JockeyName',
                   'CHAMPIONSHIPTYPE': 'ChampionshipType',
                   'FULLNAME': 'OwnerName'}, inplace=True)
dfResults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14996 entries, 0 to 64932
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Region                        14996 non-null  object 
 1   Distance                      14996 non-null  int64  
 2   Category                      14996 non-null  object 
 3   MajorEvent                    14996 non-null  float64
 4   GroundCondition               14996 non-null  float64
 5   Stick                         14996 non-null  float64
 6   HorseName                     14996 non-null  object 
 7   StartingStall                 14996 non-null  int64  
 8   Weight                        14996 non-null  int64  
 9   JockeyName                    14996 non-null  object 
 10  ChampionshipType              14996 non-null  object 
 11  OwnerName                     14996 non-null  object 
 12  TOP1                          14996 non-null  int64  
 13  T

## Guardar el dataset

In [19]:
pd.DataFrame.to_csv(dfResults, "../data/winning_horse_limpieza.csv", index=False)